In [1]:

import math, json, os, sys

import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing import image
import time

import sys
sys.path.append('../')
from Utilities.model_visualization import model_to_png


print('Imports Done')

C:\Users\Javapocalypse\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Imports Done


In [5]:
# paths to training and validation data

train_data_dir = 'dataset/train'
validation_data_dir = 'dataset/test'

In [6]:
# Params for CNN

# img_width, img_height = 150, 150
# batch_size = 5
# epochs = 200
# train_samples = 420
# validation_samples = 80
# img_channels = 3
# cardinality = 32

In [8]:
DATA_DIR = 'dataset_new'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
VALID_DIR = os.path.join(DATA_DIR, 'test')
SIZE = (224, 224)
BATCH_SIZE = 50

In [9]:
num_train_samples = sum([len(files) for r, d, files in os.walk(TRAIN_DIR)])
num_valid_samples = sum([len(files) for r, d, files in os.walk(VALID_DIR)])

In [10]:
num_train_steps = math.floor(num_train_samples/BATCH_SIZE)
num_valid_steps = math.floor(num_valid_samples/BATCH_SIZE)

In [11]:
gen = keras.preprocessing.image.ImageDataGenerator()
val_gen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, vertical_flip=True)

In [12]:
batches = gen.flow_from_directory(
    TRAIN_DIR, 
    target_size=SIZE, 
    class_mode='categorical', 
    shuffle=True, 
    batch_size=BATCH_SIZE)

val_batches = val_gen.flow_from_directory(
    VALID_DIR, 
    target_size=SIZE, 
    class_mode='categorical', 
    shuffle=True, 
    batch_size=BATCH_SIZE)

Found 9013 images belonging to 3 classes.
Found 2324 images belonging to 3 classes.


In [13]:
model = keras.applications.resnet50.ResNet50()

In [14]:
classes = list(iter(batches.class_indices))

model.layers.pop()

for layer in model.layers:
    layer.trainable=False
    
last = model.layers[-1].output

x = Dense(len(classes), activation="softmax")(last)

finetuned_model = Model(model.input, x)

finetuned_model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

for c in batches.class_indices:
    classes[batches.class_indices[c]] = c
    
finetuned_model.classes = classes

model_to_png(finetuned_model, 'Fine_Tuned_Resnet')

In [15]:
early_stopping = EarlyStopping(patience=3)

checkpointer = ModelCheckpoint('resnet50_best.h5', verbose=1, save_best_only=True, monitor='val_acc')

In [16]:
start_time = time.time()

finetuned_model.fit_generator(
    batches, 
    steps_per_epoch=num_train_steps, 
    epochs=100, 
    callbacks=[early_stopping, checkpointer], 
    validation_data=val_batches, 
    validation_steps=num_valid_steps)

print("Training Time: ", (time.time() - start_time))


finetuned_model.save('resnet50_final.h5')

Epoch 1/100
180/180 [==============================] - 286s 2s/step - loss: 0.8071 - acc: 0.6575 - val_loss: 1.3187 - val_acc: 0.4722

Epoch 00001: val_acc improved from -inf to 0.47217, saving model to resnet50_best.h5
Epoch 2/100
180/180 [==============================] - 136s 756ms/step - loss: 0.6191 - acc: 0.7576 - val_loss: 1.3467 - val_acc: 0.4757

Epoch 00002: val_acc improved from 0.47217 to 0.47565, saving model to resnet50_best.h5
Epoch 3/100
180/180 [==============================] - 136s 756ms/step - loss: 0.5716 - acc: 0.7773 - val_loss: 1.3062 - val_acc: 0.4865

Epoch 00003: val_acc improved from 0.47565 to 0.48652, saving model to resnet50_best.h5
Epoch 4/100
180/180 [==============================] - 135s 749ms/step - loss: 0.5402 - acc: 0.7902 - val_loss: 1.3711 - val_acc: 0.4822

Epoch 00004: val_acc did not improve
Epoch 5/100
180/180 [==============================] - 135s 750ms/step - loss: 0.5248 - acc: 0.7969 - val_loss: 1.2434 - val_acc: 0.5113

Epoch 00005: va